In [1]:
import pandas as pd
import urllib.parse

from sqlalchemy import create_engine

# Informasi koneksi ke PostgreSQL
username = "ftde01"
password = "ftde01!@#"
host = "34.50.87.186"
port = "5432"
database = "stream_processing"
password = urllib.parse.quote_plus(password)

# URL koneksi ke PostgreSQL
db_url = f"postgresql://{username}:{password}@{host}:{port}/{database}"
engine = create_engine(db_url)

In [2]:
df = pd.read_sql_query('SELECT * FROM old_information;', engine)
df.head(5)

,nameOrig,oldbalanceOrg,nameDest,oldbalanceDest
0,C1231006815,170136.0,M1979787155,0.0
1,C1666544295,21249.0,M2044282225,0.0
2,C1305486145,181.0,C553264065,0.0
3,C840083671,181.0,C38997010,21182.0
4,C2048537720,41554.0,M1230701703,0.0


In [3]:
df.columns

Index(['nameOrig', 'oldbalanceOrg', 'nameDest', 'oldbalanceDest'], dtype='object')

In [14]:
from pymongo import MongoClient
import pandas as pd

# Menghubungkan ke MongoDB
mongo_client = MongoClient("mongodb://localhost:27017/")
db = mongo_client["ftde01"]
collection = db["project4-collection"]

# Mengambil data dari koleksi MongoDB
data = list(collection.find())

# Memasukkan data ke dalam DataFrame Pandas
produder = pd.DataFrame(data)

produder

,_id,step,type,amount,nameOrig,newbalanceOrig,nameDest,newbalanceDest
0,66913cb197376ebdb144e888,1,PAYMENT,9839.64,C1231006815,160296.36,M1979787155,0.00
1,66913cb197376ebdb144e889,1,PAYMENT,1864.28,C1666544295,19384.72,M2044282225,0.00
2,66913cb197376ebdb144e88a,1,TRANSFER,181.00,C1305486145,0.00,C553264065,0.00
3,66913cb197376ebdb144e88b,1,CASH_OUT,181.00,C840083671,0.00,C38997010,0.00
4,66913cb197376ebdb144e88c,1,PAYMENT,11668.14,C2048537720,29885.86,M1230701703,0.00
...,...,...,...,...,...,...,...,...
116,6691409797376ebdb144e8fc,1,PAYMENT,3705.83,C671596011,38197.63,M1925352804,0.00
117,669140a197376ebdb144e8fd,1,CASH_OUT,419801.40,C1687354037,0.00,C33524623,1517262.16
118,669140ab97376ebdb144e8fe,1,CASH_OUT,335416.51,C743778731,0.00,C575335780,52415.15
119,669140b597376ebdb144e8ff,1,PAYMENT,3372.29,C967323951,38025.71,M1600594643,0.00


In [22]:
# Menghitung duplikasi dalam df
dupes_count_df = df.duplicated(subset=['nameOrig', 'nameDest']).sum()
print(f"Jumlah duplikasi di df: {dupes_count_df}")


Jumlah duplikasi di df: 45000


In [23]:
# Menghitung duplikasi dalam produder
dupes_count_produder = produder.duplicated(subset=['nameOrig', 'nameDest']).sum()
print(f"Jumlah duplikasi di produder: {dupes_count_produder}")


Jumlah duplikasi di produder: 0


In [24]:
# Menghapus duplikasi dalam df
df_cleaned = df.drop_duplicates(subset=['nameOrig', 'nameDest'])


In [29]:
data = produder.merge(df_cleaned, how='inner', on=['nameOrig','nameDest'])
predict = data.drop(['nameOrig','nameDest'], axis=1)
predict = predict.to_dict('index')[0]
predict

{'_id': ObjectId('66913cb197376ebdb144e888'),
 'step': 1,
 'type': 'PAYMENT',
 'amount': 9839.64,
 'newbalanceOrig': 160296.36,
 'newbalanceDest': 0.0,
 'oldbalanceOrg': 170136.0,
 'oldbalanceDest': 0.0}

In [30]:
predict

{'_id': ObjectId('66913cb197376ebdb144e888'),
 'step': 1,
 'type': 'PAYMENT',
 'amount': 9839.64,
 'newbalanceOrig': 160296.36,
 'newbalanceDest': 0.0,
 'oldbalanceOrg': 170136.0,
 'oldbalanceDest': 0.0}

### **Precit Fraud Detection**

In [31]:
from modelling import FraudModel

In [32]:
#parameter inputan path
import os

path = os.getcwd()
path = path + "\\modelling\\"
path

'd:\\Data-Engineer-Skola\\Task\\project\\New folder\\DE - STREAM PROCESSING\\DE - STREAM PROCESSING\\modelling\\'

In [33]:
result = FraudModel.runModel(predict, path)


d:\Data-Engineer-Skola\Task\project\New folder\DE - STREAM PROCESSING\env\Lib\site-packages\sklearn\base.py:376: InconsistentVersionWarning: Trying to unpickle estimator OneHotEncoder from version 1.3.0 when using version 1.5.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
d:\Data-Engineer-Skola\Task\project\New folder\DE - STREAM PROCESSING\env\Lib\site-packages\sklearn\base.py:376: InconsistentVersionWarning: Trying to unpickle estimator StandardScaler from version 1.3.0 when using version 1.5.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
d:\Data-Engineer-Skola\Task\project\New folder\DE - STREAM PROCESSING\env\Lib\site-packages\sklearn\base.py:376

In [34]:
data['predict'] = result
data

,_id,step,type,amount,nameOrig,newbalanceOrig,nameDest,newbalanceDest,oldbalanceOrg,oldbalanceDest,predict
0,66913cb197376ebdb144e888,1,PAYMENT,9839.64,C1231006815,160296.36,M1979787155,0.00,170136.00,0.0,White List
1,66913cb197376ebdb144e889,1,PAYMENT,1864.28,C1666544295,19384.72,M2044282225,0.00,21249.00,0.0,White List
2,66913cb197376ebdb144e88a,1,TRANSFER,181.00,C1305486145,0.00,C553264065,0.00,181.00,0.0,White List
3,66913cb197376ebdb144e88b,1,CASH_OUT,181.00,C840083671,0.00,C38997010,0.00,181.00,21182.0,White List
4,66913cb197376ebdb144e88c,1,PAYMENT,11668.14,C2048537720,29885.86,M1230701703,0.00,41554.00,0.0,White List
...,...,...,...,...,...,...,...,...,...,...,...
116,6691409797376ebdb144e8fc,1,PAYMENT,3705.83,C671596011,38197.63,M1925352804,0.00,41903.46,0.0,White List
117,669140a197376ebdb144e8fd,1,CASH_OUT,419801.40,C1687354037,0.00,C33524623,1517262.16,38197.63,499962.0,White List
118,669140ab97376ebdb144e8fe,1,CASH_OUT,335416.51,C743778731,0.00,C575335780,52415.15,144478.00,295.0,White List
119,669140b597376ebdb144e8ff,1,PAYMENT,3372.29,C967323951,38025.71,M1600594643,0.00,41398.00,0.0,White List


In [35]:
data

,_id,step,type,amount,nameOrig,newbalanceOrig,nameDest,newbalanceDest,oldbalanceOrg,oldbalanceDest,predict
0,66913cb197376ebdb144e888,1,PAYMENT,9839.64,C1231006815,160296.36,M1979787155,0.00,170136.00,0.0,White List
1,66913cb197376ebdb144e889,1,PAYMENT,1864.28,C1666544295,19384.72,M2044282225,0.00,21249.00,0.0,White List
2,66913cb197376ebdb144e88a,1,TRANSFER,181.00,C1305486145,0.00,C553264065,0.00,181.00,0.0,White List
3,66913cb197376ebdb144e88b,1,CASH_OUT,181.00,C840083671,0.00,C38997010,0.00,181.00,21182.0,White List
4,66913cb197376ebdb144e88c,1,PAYMENT,11668.14,C2048537720,29885.86,M1230701703,0.00,41554.00,0.0,White List
...,...,...,...,...,...,...,...,...,...,...,...
116,6691409797376ebdb144e8fc,1,PAYMENT,3705.83,C671596011,38197.63,M1925352804,0.00,41903.46,0.0,White List
117,669140a197376ebdb144e8fd,1,CASH_OUT,419801.40,C1687354037,0.00,C33524623,1517262.16,38197.63,499962.0,White List
118,669140ab97376ebdb144e8fe,1,CASH_OUT,335416.51,C743778731,0.00,C575335780,52415.15,144478.00,295.0,White List
119,669140b597376ebdb144e8ff,1,PAYMENT,3372.29,C967323951,38025.71,M1600594643,0.00,41398.00,0.0,White List


In [36]:
from modelling import FraudModel
import os

# Parameter inputan path
path = os.getcwd()
path = path + "\\modelling\\"
print("Path:", path)

# Menjalankan model
result = FraudModel.runModel(predict, path)
print("Result:", result)
print("Result type:", type(result))
print("Result length:", len(result))




Path: d:\Data-Engineer-Skola\Task\project\New folder\DE - STREAM PROCESSING\DE - STREAM PROCESSING\modelling\
Result: White List
Result type: <class 'str'>
Result length: 10


d:\Data-Engineer-Skola\Task\project\New folder\DE - STREAM PROCESSING\env\Lib\site-packages\sklearn\base.py:376: InconsistentVersionWarning: Trying to unpickle estimator OneHotEncoder from version 1.3.0 when using version 1.5.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
d:\Data-Engineer-Skola\Task\project\New folder\DE - STREAM PROCESSING\env\Lib\site-packages\sklearn\base.py:376: InconsistentVersionWarning: Trying to unpickle estimator StandardScaler from version 1.3.0 when using version 1.5.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
d:\Data-Engineer-Skola\Task\project\New folder\DE - STREAM PROCESSING\env\Lib\site-packages\sklearn\base.py:376

In [38]:
from pymongo import MongoClient

# Mengatur koneksi ke MongoDB lokal tanpa autentikasi
mongo_client = MongoClient("mongodb://localhost:27017/")

db = mongo_client["ftde-01-result"]
collection = db["project4-result"]

# Asumsikan 'data' adalah DataFrame pandas yang sudah didefinisikan sebelumnya

for _, row in data.iterrows():
    document = row.to_dict()
    collection.insert_one(document)

print("Data telah disimpan ke MongoDB")

Data telah disimpan ke MongoDB
